# Gas grid simulation setup
## Creating an empty co-simulation graph

In [1]:
from zerobnl import CoSim

sim = CoSim()

## Read Gas grid from database

In [2]:
from dblayer import *

connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
    )

In [3]:
from dblayer.sim.pandangas import *

pg_reader = PandaNGasModelDBReader( connect )

In [4]:
net = pg_reader.get_net( network_id = 3000 )
load_names = [ load[ 'name' ] for index, load in net.load.iterrows() ]

In [5]:
print(net)

This pandangas network includes the following parameter tables:
   - pipe (287 elements)
   - bus (277 elements)
   - load (123 elements)
   - feeder (2 elements)


## Simulation setup for gas grid

The gas grid is simulated with the help of package [PandaNGas](https://github.com/IntegrCiTy/PandaNGas).

In [6]:
import os.path

set_attrs = [ ( n + '/p_kW', 'kW' ) for n in load_names ]

sim.create_meta_model(
    meta_model = 'GasGridMeta',
    list_of_attrs_to_set = set_attrs,
    list_of_attrs_to_get = []
)

sim.create_environment(
    env = 'GasGridEnv', 
    wrapper = os.path.join( 'wrappers', 'wrapper_gasgrid.py' ),
    dockerfile = os.path.join( 'dockerfiles', 'Dockerfile_gasgrid' )
)

sim.add_node(
    node = 'GasGrid', 
    meta = 'GasGridMeta',
    env = 'GasGridEnv',
    init_values = {   
        'dbuser': 'postgres',
        'dbpwd': 'postgres',
        'dbhost': 'citydb-container',
        'dbname': 'citydb',
        'dbport': 5432,
        'network_id': 3000
    }
)

## Simulation setup for consumers

In [7]:
db_access = DBAccess()
db_access.connect_to_citydb(connect)


FinalEnergy = db_access.map_citydb_object_class( 'FinalEnergy', table_name='nrg8_final_energy_ts', schema='citydb_view' )

ConvSystemToFinalEnergy = db_access.map_citydb_object_class( 'ConvSystemToFinalEnergy',
    table_name='nrg8_conv_sys_to_final_nrg', schema='citydb', user_defined = True
    )

Boiler = db_access.map_citydb_object_class( 'Boiler', table_name='nrg8_conv_system', schema='citydb' )

TerminalElement = db_access.map_citydb_object_class( 'TerminalElement', table_name='utn9_ntw_feat_term_elem', schema='citydb_view' )

NetworkToFeature = db_access.map_citydb_object_class( 'NetworkToFeature',
    table_name='utn9_network_to_network_feature', schema='citydb', user_defined = True
    )

final_energy_db_data = db_access.join_citydb_objects(
    [ 'FinalEnergy', 'TerminalElement' ], 
    conditions = [
        FinalEnergy.id == ConvSystemToFinalEnergy.final_nrg_id,
        ConvSystemToFinalEnergy.conv_system_id == Boiler.id,
        Boiler.inst_in_ctyobj_id == TerminalElement.conn_cityobject_id,
        TerminalElement.id == NetworkToFeature.network_feature_id,
        NetworkToFeature.network_id == 3000        
        ]
    )

In [8]:
gas_demand = { data[1].name: data[0].ts_values_array for data in final_energy_db_data }
time_series_start_date = final_energy_db_data[0][0].ts_temporal_extent_begin
time_series_interval = final_energy_db_data[0][0].ts_time_interval
time_series_interval_unit = final_energy_db_data[0][0].ts_time_interval_unit
time_series_length = final_energy_db_data[0][0].ts_array_length

In [9]:
import pandas as pd

df = pd.DataFrame.from_dict(gas_demand)

date = pd.date_range(
    start = time_series_start_date, 
    periods = time_series_length, 
    freq = '{length}{unit}'.format( length=time_series_interval, unit=time_series_interval_unit )
)

df['date'] = date.values
df.set_index('date', inplace=True)

In [10]:
df.to_pickle( './gas_demand.pkl' )

In [11]:
from dblayer.zerobnl.associate import *
time_series = final_energy_db_data[0][0]
start_date = AssociateCityDBObjectAttribute( time_series, 'ts_temporal_extent_begin' )

In [12]:
get_attrs = [ ( n, 'kW' ) for n in load_names ]

sim.create_meta_model(
    meta_model = 'ConsumersMeta',
    list_of_attrs_to_set = [],
    list_of_attrs_to_get = get_attrs
)

sim.create_environment(
    env = 'ConsumersEnv', 
    wrapper = os.path.join( 'wrappers', 'wrapper_consumers.py' ),
    dockerfile = os.path.join( 'dockerfiles', 'Dockerfile_consumers' )
)

sim.add_node(
    node = 'Consumers', 
    meta = 'ConsumersMeta',
    env = 'ConsumersEnv',
    init_values = { 'start_date': start_date },
    files = [ './gas_demand.pkl' ]
)

## Links

In [13]:
for load in load_names:
    sim.add_link( 'Consumers', load, 'GasGrid', '{}/p_kW'.format( load ) )

In [14]:
sim.links

,GetNode,GetAttr,SetNode,SetAttr,Unit
0,Consumers,gas_node47,GasGrid,gas_node47/p_kW,kW
1,Consumers,gas_node48,GasGrid,gas_node48/p_kW,kW
2,Consumers,gas_node49,GasGrid,gas_node49/p_kW,kW
3,Consumers,gas_node50,GasGrid,gas_node50/p_kW,kW
4,Consumers,gas_node51,GasGrid,gas_node51/p_kW,kW
...,...,...,...,...,...
118,Consumers,gas_node166,GasGrid,gas_node166/p_kW,kW
119,Consumers,gas_node167,GasGrid,gas_node167/p_kW,kW
120,Consumers,gas_node168,GasGrid,gas_node168/p_kW,kW
121,Consumers,gas_node274,GasGrid,gas_node274/p_kW,kW


## Simulation parameters

In [15]:
sim.create_sequence( [ [ 'Consumers' ], [ 'GasGrid' ] ] )
sim.set_time_unit( 'seconds' )
sim.create_steps( 24 * [60*60] )

## Save the simulation setup to the database

In [16]:
#db_access.cleanup_simpkg_schema()

In [17]:
from dblayer.zerobnl.writer import *

writer = DBWriter( connect )
writer.write_to_db( sim, 'GasGridSim', write_meta_models = True, write_envs = True )

In [18]:
del writer